In [6]:
from pyrosetta import *
from pyrosetta.toolbox import *
from pyrosetta.teaching import *
from python_scripts.pyrosetta_additional_scripts import predict_ddG
import os
init()

PyRosetta-4 2023 [Rosetta PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python39.Release 2023.27+release.e3ce6ea9faf661ae8fa769511e2a9b8596417e58 2023-07-07T12:00:46] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python39.Release r353 2023.27+release.e3ce6ea e3ce6ea9faf661ae8fa769511e2a9b8596417e58 http://www.pyrosetta.org 2023-07-07T12:00:46
core.init: command: PyRosetta -ex1 -ex2aro -database /central/groups/MazmanianLab/joeB/software/mambaforge/envs/pyrosetta/lib/python3.9/site-packages/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-325964077 seed_offset=0 real_seed=-325964077 thread_index=0
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=-3259

In [7]:

# Load the two protein structures
# pose1 = pose_from_pdb("../6X93_A.pdbqt")
# pose2 = pose_from_pdb("../6X93_B.pdbqt")
pose = pose_from_rcsb("6X93")


core.import_pose.import_pose: File '6X93.clean.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue LYS 100
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue LYS 100
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CE  on residue LYS 100
core.conformation.Conformation: [ WARNING ] missing heavyatom:  NZ  on residue LYS 100
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue ASN 101
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OD1 on residue ASN 101
core.conformation.Conformation: [ WARNING ] missing heavyatom:  ND2 on residue ASN 101
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue PHE 103
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD1 on residue PHE 103
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD2 on residue PHE 103
core.conformation.Conformation: [ W

In [8]:
res_24 = pose.residue(24)
print(res_24)
print(res_24.name())
print(res_24.is_charged())

Residue 24: THR (THR, T):
Base: THR
 Properties: POLYMER PROTEIN CANONICAL_AA SC_ORBITALS POLAR BETA_BRANCHED_SIDECHAIN METALBINDING ALPHA_AA L_AA
 Variant types:
 Main-chain atoms:  N    CA   C  
 Backbone atoms:    N    CA   C    O    H    HA 
 Side-chain atoms:  CB   OG1  CG2  HB   HG1 1HG2 2HG2 3HG2
Atom Coordinates:
   N  : 142.43, 118.78, 107.888
   CA : 142.205, 117.592, 107.072
   C  : 142.976, 116.398, 107.599
   O  : 142.558, 115.255, 107.404
   CB : 142.588, 117.861, 105.626
   OG1: 142.05, 119.124, 105.221
   CG2: 142.042, 116.777, 104.73
   H  : 142.83, 119.603, 107.46
   HA : 141.145, 117.341, 107.111
   HB : 143.674, 117.889, 105.536
   HG1: 141.569, 119.516, 105.954
  1HG2: 142.324, 116.982, 103.697
  2HG2: 142.451, 115.814, 105.034
  3HG2: 140.956, 116.751, 104.811
Mirrored relative to coordinates in ResidueType: FALSE

THR
False


In [9]:
resid = pose.pdb_info().pdb2pose('A', 28)
print("phi:", pose.phi(resid))
print("psi:", pose.psi(resid))
print("chi1:", pose.chi(1, resid))

phi: -71.19543965318766
psi: -29.00462311389512
chi1: 63.77191195491151


In [10]:
print(pose.residue(17))

Residue 17: ASP (ASP, D):
Base: ASP
 Properties: POLYMER PROTEIN CANONICAL_AA SC_ORBITALS POLAR CHARGED NEGATIVE_CHARGE METALBINDING ALPHA_AA L_AA
 Variant types:
 Main-chain atoms:  N    CA   C  
 Backbone atoms:    N    CA   C    O    H    HA 
 Side-chain atoms:  CB   CG   OD1  OD2 1HB  2HB 
Atom Coordinates:
   N  : 140.962, 130.038, 109.28
   CA : 141.502, 129.604, 108.008
   C  : 142.894, 129.013, 108.161
   O  : 143.274, 128.144, 107.374
   CB : 141.454, 130.749, 107.001
   CG : 142.315, 131.918, 107.387
   OD1: 143.205, 131.763, 108.238
   OD2: 142.078, 133.02, 106.849
   H  : 140.874, 131.027, 109.466
   HA : 140.891, 128.781, 107.635
  1HB : 141.779, 130.389, 106.025
  2HB : 140.426, 131.098, 106.895
Mirrored relative to coordinates in ResidueType: FALSE



In [11]:
# Double Check: does resid contain the Pose numbering or PDB numbering?
res_28 = pose.residue(resid)
N28 = AtomID(res_28.atom_index("N"), resid)
CA28 = AtomID(res_28.atom_index("CA"), resid)
C28 = AtomID(res_28.atom_index("C"), resid)
print(CA28)

 atomno= 2 rsd= 17 


In [12]:
res_28.atom_index("C")
# ?AtomID

3

In [14]:
import glob
import logging
logging.basicConfig(level=logging.INFO)
import numpy as np
import os
import pyrosetta
import pyrosetta.distributed
import pyrosetta.distributed.io as io
import pyrosetta.distributed.viewer as viewer
import sys

In [15]:
view = viewer.init(pose, window_size=(800, 600)) + viewer.setStyle()
view()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [16]:

pose = pose_from_rcsb("2FD7")
# pose = pyrosetta.io.pose_from_rcsb("2FD7")

core.import_pose.import_pose: File '2FD7.clean.pdb' automatically determined to be of type PDB


core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG1 on residue VAL 15
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG2 on residue VAL 15
core.conformation.Conformation: Found disulfide between residues 3 40
core.conformation.Conformation: Found disulfide between residues 4 32
core.conformation.Conformation: Found disulfide between residues 16 26
core.pack.pack_missing_sidechains: packing residue number 15 because of missing atom number 6 atom name  CG1
core.pack.task: Packer task: initialize from command line()
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.pack.pack_rotamers: built 1 rotamers at 1 positions.
core.pack.pack_rotamers: Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
core.pack.rotamer_set.RotamerSets: Completed interaction graph pre-calculation in 1 available threads (1 had been requested).


In [17]:
view = viewer.init(pose, delay=0.15) \
    + viewer.setStyle(radius=0.1) \
    + viewer.setDisulfides(radius=0.1)
backrub = pyrosetta.rosetta.protocols.backrub.BackrubMover()
minimize = pyrosetta.rosetta.protocols.minimization_packing.MinMover()
view()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [19]:
for _ in range(100):
    backrub.apply(pose)
    minimize.apply(pose)
    view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [89]:
pose = pyrosetta.Pose()
view = viewer.init(pose, delay=0.5) + viewer.setStyle(radius=0.1)

add_pdb_info_mover = pyrosetta.rosetta.protocols.simple_moves.AddPDBInfoMover()
mb = pyrosetta.rosetta.protocols.helical_bundle.MakeBundle()
bpc = pyrosetta.rosetta.protocols.helical_bundle.BPC_Parameters

n_helices = 6
for h in range(1, (n_helices + 1)):
    mb.add_helix()
    mb.helix(h).calculator_op().real_parameter(bpc.BPC_delta_omega0).set_value(h * (np.radians(360) / n_helices))
for i in range(2, 42, 3):
    for h in range(1, (n_helices + 1)):
        mb.helix(h).set_helix_length(i)
        mb.helix(h).calculator_op().real_parameter(bpc.BPC_r0).set_value(n_helices * 2)
        mb.helix(h).calculator_op().real_parameter(bpc.BPC_omega0).set_value(np.radians(5))
    mb.apply(pose)
    add_pdb_info_mover.apply(pose)
    view()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [91]:
# pdbfiles = [os.path.join("inputs", f) for f in ["3EK4.pdb", "6MSR.pdb", "1QCQ.pdb", "2FD7.pdb"]]
poses = [pose_from_rcsb(p) for p in ["3EK4", "6MSR", "1QCQ", "2FD7"]]

core.import_pose.import_pose: File '3EK4.clean.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue ASN:CtermProteinFull 348


core.import_pose.import_pose: File '6MSR.clean.pdb' automatically determined to be of type PDB
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 1 in file 6MSR.clean.pdb. Best match rsd_type:  GLY:NtermProteinFull
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 77 in file 6MSR.clean.pdb. Best match rsd_type:  GLY:NtermProteinFull
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 153 in file 6MSR.clean.pdb. Best match rsd_type:  GLY:NtermProteinFull
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue GLU 3
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OE1 on residue GLU 3
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OE2 on residue GLU 3
core.conformation.Conformation: [ WARNING ] missing heavyatom:  NE  on residue ARG 7
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CZ  on residue ARG 7
core.conformati

In [96]:
viewer.presets.coreBoundarySurface(poses, window_size=(800, 600), continuous_update=True)
# viewer.presets.ligandsAndMetals(poses, window_size=(800, 600), continuous_update=False)

core.select.residue_selector.LayerSelector: Setting LayerSelector to use sidechain neighbors to determine burial.
core.select.residue_selector.LayerSelector: Set cutoffs for core and surface to 5.2 and 2, respectively, in LayerSelector.
core.select.residue_selector.LayerSelector: Setting core=true boundary=false surface=false in LayerSelector.
core.select.residue_selector.LayerSelector: Setting LayerSelector to use sidechain neighbors to determine burial.
core.select.residue_selector.LayerSelector: Set cutoffs for core and surface to 5.2 and 2, respectively, in LayerSelector.
core.select.residue_selector.LayerSelector: Setting core=false boundary=true surface=false in LayerSelector.
core.select.residue_selector.LayerSelector: Setting LayerSelector to use sidechain neighbors to determine burial.
core.select.residue_selector.LayerSelector: Set cutoffs for core and surface to 5.2 and 2, respectively, in LayerSelector.
core.select.residue_selector.LayerSelector: Setting core=false boundary

interactive(children=(IntSlider(value=0, description='Decoys', max=3), Output()), _dom_classes=('widget-intera…

<function pyrosetta.distributed.viewer.core.Viewer.show.<locals>.view(i=0)>

In [95]:
# import pyrosetta.distributed.viewer as viewer
# viewer.presets.myCustomPreset(poses)

AttributeError: module 'pyrosetta.distributed.viewer.presets' has no attribute 'myCustomPreset'

In [6]:
# Create a franklin2019 energy function
sfxn = create_score_function("franklin2019")

protocols.membrane.scoring.FaWaterToBilayerEnergy: Reading fa_water_to_bilayer parameters from the database
basic.io.database: Database file opened: membrane/memb_fa_params_2019.txt


In [ ]:
# Repack and score the native conformation
reference_pose = predict_ddG.mutate_residue(pose1, 104, "A", 8.0, sfxn)


In [1]:

# Repack and score the native conformation
# reference_pose = predict_ddG.mutate_residue(pose1, 104, "A", 8.0, sfxn)
# reference_pose = predict_ddG.compute_ddG(pose1, sfxn, 104, "A", 8.0, sfxn)

# Score the native pose and grab the native AA
native_score = sfxn( pose1)
native_score
# native_aa = pose.residue( resnum ).name1()

NameError: name 'create_score_function' is not defined

In [34]:
?predict_ddG


Type:        module
String form: <module 'python_scripts.pyrosetta_additional_scripts.predict_ddG' from '/central/groups/Mazmanian <...> icrobiota-Immunomodulation/notebooks/python_scripts/pyrosetta_additional_scripts/predict_ddG.py'>
File:        /central/groups/MazmanianLab/joeB/Microbiota-Immunomodulation/Microbiota-Immunomodulation/notebooks/python_scripts/pyrosetta_additional_scripts/predict_ddG.py
Docstring:  
Predict the ddG of mutation

This script computes the energetic cost of single point
mutations in membrane proteins. To account for the conformational
change, we repack side chains within 8 Angstroms of the host site.
This script generates data for Test 7.

Authors: 
    Rebecca Alford <ralford3@jhu.edu> 

Example: 
    $ import predict_ddG
    $ predict_ddG.run_ddG_of_mutation_calc( 
    config, energy_fxn, targets_dir, native_pdb, 
    native_span, reference_data )
        
Arguments: 
    - config: Container with path to benchmark and rosetta files
    - energy_fxn: Weig

In [28]:
# Identify the interface residues
# interface_residues = compute_interface_residues(pose1, pose2)
interface_residues = pyrosetta.rosetta.protocols.interface.select_interface_residues(pose1)
interface_residues

TypeError: select_interface_residues(): incompatible function arguments. The following argument types are supported:
    1. (pose: pyrosetta.rosetta.core.pose.Pose, interface: str, interface_distance: int) -> pyrosetta.rosetta.utility.vector1_bool

Invoked with: <pyrosetta.rosetta.core.pose.Pose object at 0x7fc1e5d84fb0>

In [ ]:
# Calculate the RMSD between the interface residues
rmsd = calculate_rmsd(pose1.select_by_residue_range(interface_residues), pose2.select_by_residue_range(interface_residues))

# Print the RMSD
print(rmsd)

In [23]:
print(dir(pyrosetta))
print(dir(pyrosetta.Pose))


['AtomID', 'CD', 'EnergyMethod', 'FoldTree', 'LoggingContext', 'MonteCarlo', 'MoveMap', 'Pose', 'PyJobDistributor', 'PyMOLMover', 'PyRosettaException', 'PythonPyExitCallback', 'RepeatMover', 'ScoreFunction', 'ScoreTypesRegistry', 'SequenceMover', 'Set', 'SwitchResidueTypeSetMover', 'TrialMover', 'Vector1', '_ScoreTypesRegistryByType_', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_is_interactive', '_mem_EnergyCreators_', '_mem_EnergyMethods_', '_rosetta_database_from_env', '_vector_extend_func', '_version_string', 'absolute_import', 'bindings', 'create_score_function', 'defineEnergyMethodCreator', 'distributed', 'dump_pdb', 'etable_atom_pair_energies', 'generate_nonstandard_residue_set', 'get_fa_scorefxn', 'get_score_function', 'init', 'io', 'k', 'logger', 'logging', 'logging_support', 'namedtuple', 'os', 'pose_from_file', 'pose_from_pdb', 'pose_from_sequence', 'poses_from_files', 'poses_from_sequences', 'poses_f

In [25]:
?pyrosetta.rosetta.protocols.interface.select_interface_residues

Docstring:
select_interface_residues(pose: pyrosetta.rosetta.core.pose.Pose, interface: str, interface_distance: int) -> pyrosetta.rosetta.utility.vector1_bool

Get a set of interface residues using the dock_chains interface string: Ex: LH_A.
 

 Uses Steven's InterGroupNeighborsCalculator.  Does not require pose to have specific foldtree.

C++: protocols::interface::select_interface_residues(const class core::pose::Pose &, std::string, unsigned long) --> class utility::vector1<bool, class std::allocator<bool> >
Type:      builtin_function_or_method